# Step 2: Vector DBで類似検索

本ステップでは、ひとつ前のステップで構造化データを登録したベクトルデータベースから、サンプルのクエリを投入して類似検索を実行する過程を経験します。
- Hugging FaceからEmbedding Modelを取得してメモリに読み込み、Modelを指定してベクトルデータベースに接続する
- 類似検索実行前に「pkカラム」で条件指定してベクトルデータベースへ検索の疎通確認をする
- サンプルのクエリを用意していくつかのオプション指定で類似検索を試す
![Step2](../image/rag-overview-step2.png)

## 0. 事前準備

### 共通処理/定数定義
全ステップで共通して使用する定数とバナークラスを読み込む

In [ ]:
from mylib import myconstant
from mylib.MyBanner import MyBanner

### パッケージインストール
本ステップの処理で依存するパッケージをインストールする

In [ ]:
MyBanner.start()

!python -V
!pip install langchain
!pip install langchain-huggingface
!pip install langchain-milvus

!pip install ipywidgets
!pip install urllib3==1.26.20

MyBanner.finish()

### import
本ステップの処理で依存するモジュールを読み込む

In [ ]:
MyBanner.start()

from mylib.MyEmbedding import MyEmbedding
from mylib.MyMilvus import MyMilvus

MyBanner.finish()

## 1. 生成

### 【準備】Embedding Model読込
Embedding Modelをメモリに読み込む

In [ ]:
MyBanner.start()

embeddings = MyEmbedding.get_model()
print(f"{embeddings=}")

MyBanner.finish()

### 【準備】Vector DB接続
エンベディングで利用するEmbedding Modelと接続情報を渡してベクトルデータベースへ接続する
- 接続失敗した場合は、Milvus(milvus-standalone コンテナ)が起動しているか確認する

In [ ]:
MyBanner.start()

# connect to VectorDB
vector_db = MyMilvus(
    myconstant.VDB_HOST, myconstant.VDB_PORT,
    myconstant.VDB_USER, myconstant.VDB_PASS, embeddings)
print(f"{vector_db=}")

MyBanner.finish()

### 【準備】Doc Store接続
RDBのテーブルに該当するドキュメントストアに接続する

In [ ]:
MyBanner.start()

# connect to a store in Vector DB
docstore_list = vector_db.get_collections()
docstore_name = docstore_list[0]
docstore = vector_db.connect(docstore_name)
print(f"{docstore_list=}")
print(f"{docstore_name=}")
print(f"{docstore=}")

MyBanner.finish()

### 【確認】類似検索前にVectorDB検索確認
類似検索を試す前にフィールドで条件指定した検索で、ベクトルデータベースの接続と検索動作を確認する

In [ ]:
MyBanner.start()
from pymilvus import MilvusClient
import pandas as pd

pk_list = docstore.get_pks(expr = "pk > 0")
print(f"{pk_list=}")
pk_list = pk_list[:4]
print(f"{pk_list=}")

connection_args = vector_db.get_connection_args()
client = MilvusClient(uri = connection_args['uri'], token = connection_args['token'])
res = client.get(
    collection_name = docstore_name,
    ids=pk_list
)

for i, milvus_rec in enumerate(res):
    res[i]['vector'] = "["+", ".join(map(str, milvus_rec['vector']))+"]"
    res[i]['pk'] = str(milvus_rec['pk'])

df_s = pd.DataFrame.from_dict(res).reindex(columns=['text', 'pk', 'vector'])
display(df_s)

MyBanner.finish()

## 2. Vector DBテキスト類似検索
数種類のオプション指定を用いて類似検索を試す

### 1) similarity_search: オプションなし デフォルト

In [ ]:
MyBanner.start()

query = "パソコンの使い方を学べるセミナーを教えてください"
docs = docstore.similarity_search(query)
for doc in docs:
    print("*", {"metadata": doc.metadata, "content": doc.page_content[0:100]} )

MyBanner.finish()

### 2) similarity_search: 結果取得数をkで指定(デフォルトは4)

In [ ]:
MyBanner.start()

# 結果の取得数をkで指定(デフォルトは4)
docs = docstore.similarity_search(query, k=10)
for doc in docs:
    print("*", {"metadata": doc.metadata, "content": doc.page_content[0:100]} )

MyBanner.finish()

### 3) similarity_search_with_score: 類似度のスコアも一緒に出力

In [ ]:
MyBanner.start()

# The closer the score is to 1, the higher the similarity.
docs = docstore.similarity_search_with_score(query, k=10)
for doc, score in docs:
    print("*", {"score": score, "metadata": doc.metadata, "content": doc.page_content[0:100]} )

MyBanner.finish()

### 4) similarity_search_with_score: exprオプションで 'category' フィルター

In [ ]:
MyBanner.start()

# 一つ前の検索結果で最終レコードのcategoryでフィルターして検索
target = doc.metadata["category"]
print(f"{target=}")
docs = docstore.similarity_search_with_score(query, k=10, expr="category=='%s'" % target)
for doc, score in docs:
    print("*", {"score": score, "metadata": doc.metadata, "content": doc.page_content[0:100] + "..."} )

MyBanner.finish()

## 3. 本ステップを終えて

ここまでの手順でベクトルデータベースを使った類似検索を経験しました。次のステップではLLMに回答案を生成を依頼するためのテンプレート作成の過程を経験します。
- 次のStep ≫ [Step 3: LLM Template作成](./rag-step03-llm_template.ipynb)
- 今のStep ≫ Step 2: Vector DBで類似検索
- 前のStep ≫ [Step 1: 構造化データのVector DB登録](./rag-step01-excel_to_vectordb.ipynb)